In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import urllib.request
import time
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import geopandas as gpd
import seaborn as sns
from scipy import stats

In [38]:
total_pages = 93
links = pd.DataFrame(columns = ['link'])
done = 0

for page in range(1, total_pages+1):
    
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
    url = "https://www.zoocasa.com/hamilton-on-sold-listings?page=" + str(page)
    headers={'User-Agent':user_agent} 

    request=urllib.request.Request(url,None,headers) #The assembled request
    data_raw = urllib.request.urlopen(request).read()
    time.sleep(1)

    soup = BeautifulSoup(data_raw, 'lxml')
    
    for add in soup.find_all("span", itemprop="address"):
        
        links.loc[done] = "https://www.zoocasa.com/hamilton-on-real-estate/" + add.text.lower().replace(' ','-')
        done +=1

links.to_csv('house_links(Hamilton).csv')

In [160]:
links = pd.read_csv("house_links(Hamilton).csv")
links.shape

(2325, 2)

In [132]:
links.loc[1,'link']

'https://www.zoocasa.com/hamilton-on-real-estate/37-pisa-dr'

In [150]:
import requests

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
headers={'User-Agent':user_agent} 

url = links.loc[0,'link']
#url = "https://www.zoocasa.com/toronto-on-real-estate/6136914-1303-38-grenville-st-toronto-on-m4y1a5-vow"
#session = requests.Session()

#pw = {'email':'roxanne0407lxt@gmail.com','password':'Roxanne20001007'}

#s = session.post(url,data = pw, headers = headers)

request=urllib.request.Request(url,None,headers) #The assembled request
data_raw = urllib.request.urlopen(request).read()
soup = BeautifulSoup(data_raw, 'lxml')

#soup = BeautifulSoup(s.text, 'html.parser')


In [151]:
import json
script = soup.find("script", id="__NEXT_DATA__")
raw = str(script).split(">")[1].replace('</script',"")

json_data = json.loads(raw)
json_data

{'props': {'pageProps': {'routeName': 'listing',
   'props': {'listingData': {'id': 10486964,
     'price': 649900,
     'soldPrice': None,
     'bedrooms': 4,
     'bedroomsPartial': None,
     'bathrooms': 2,
     'bathroomsPartial': 0,
     'squareFootage': {'min': None, 'max': 1800},
     'isVow': False,
     'isCrea': True,
     'isRental': False,
     'unitNumber': None,
     'streetNumber': '89',
     'streetName': 'Barton Street W',
     'neighbourhoodName': 'Central',
     'city': 'Hamilton',
     'imageUrl': 'https://cdn2.zoocasa.com/images/listings/4bcf2e41-d859-49e6-a39f-f5ebf7a41d04.jpg',
     'thumbnailUrl': 'https://thumbnails.zoocasa.com/10486964-89-barton-street-w-hamilton-on-H4096161/photo-1.jpg',
     'addedAt': 1612361212277,
     'expiredAt': 1613314962997,
     'soldAt': None,
     'path': '/hamilton-on-real-estate/10486964-89-barton-st-w-hamilton-on-l8l1a7',
     'province': 'ON',
     'addressPath': '/hamilton-on-real-estate/89-barton-st-w',
     'status': 'not_

In [152]:
#json_data['props']['pageProps']['props']['listingsData'][0]['position']['coordinates']

[-79.8697368, 43.2649203]

In [ ]:
for i in json_data['props']['pageProps']['props']['listingsData']:
    
    print (i['streetNumber'], i['streetName'], i['city'],'\n',
           "isVow:", i['isVow'], '\n',
           "isCrea", i['isCrea'],'\n',
           "isRental", i['isRental'],'\n',
           "price:", i['price'],'\n',
           "sold price:", i['soldPrice'],'\n',
           "last status", i['lastStatus'],'\n',
           "mlsNum:", i['mlsNum'],'\n',
           "description:", i['description'],'\n',
           "type:", i['type'], '\n',
           "bedrooms:", i['bedrooms'], '\n',
           "bathrooms:", i['bathrooms'], '\n',
           "sqft:", i['squareFootage'], '\n',
           #"balcony:", i['balcony'], '\n',
           #"heatIncluded:",i['heatIncluded'],'\n',
           #"hydroIncluded:",i['hydroIncluded'],'\n',
           "type:", i['type'],'\n',
           "parking:", i['parking'],'\n',
           #"kitchens:",i['kitchens'],'\n',
           #"garageSpaces:", i['garageSpaces'],'\n',
           "taxes:", i['taxes'],'\n',
           "exterior", i['exterior'],'\n',
           "basement",i['basement'],'\n',
           "pool", i['pool'],'\n',
           "heatingFuel",i['heatingFuel'],'\n',
           "extras:", i['extras'],'\n',
           "neighbourhoodName:", i['neighbourhoodName'],'\n'
           "url", url, '\n'
           
                 
          )
    
    
#['listingData']

In [ ]:
data_HT = pd.DataFrame(columns=['Link_no','Title','Address','Sold_price','List_price','Type',
                                'Bedrooms','Bathrooms','Exterior','Basement','Pool',
                                'Sqft_max','Sqft_min','Parking','Balcony','Kitchens',
                                'Lot_frontage','Lot_depth','Parking_Included','Parking_Spaces',
                                'Heat_Included','Hydro_Included','Approx_Age','Extras','Is_rental','Description',
                                'MLS','Neighbourhood','Latitude','Longitude','Link'])
skipped = []
i = 0
num = links.shape[0]

for link_num in range(0, num):
    try:
        url = links.loc[link_num,'link']

        request=urllib.request.Request(url,None,headers)
        data_raw = urllib.request.urlopen(request).read()
        soup = BeautifulSoup(data_raw, 'lxml')

        script = soup.find("script", id="__NEXT_DATA__")
        raw = str(script).split(">")[1].replace('</script',"")
        json_data = json.loads(raw)

        for listing in json_data['props']['pageProps']['props']['listingsData']:
                
            if listing['soldPrice'] != None:
                
                data_HT.loc[i,'Link_no'] = link_num
                data_HT.loc[i,'Title'] = soup.title.contents[0]
                data_HT.loc[i,'Address'] = listing['streetNumber']+" "+listing['streetName']
                data_HT.loc[i,'Is_rental'] = listing['isRental']
                data_HT.loc[i,'Sold_price'] = listing['soldPrice']
                data_HT.loc[i,'List_price'] = listing['price']
                data_HT.loc[i,'Bedrooms'] = listing['bedrooms']
                data_HT.loc[i,'Bathrooms'] = listing['bathrooms']
                data_HT.loc[i,'Sqft_max'] = listing['squareFootage']['max']
                data_HT.loc[i,'Sqft_min'] = listing['squareFootage']['min']
                data_HT.loc[i,'Parking'] = listing['parking']
                data_HT.loc[i,'Exterior'] = listing['exterior']
                data_HT.loc[i,'Basement'] = listing['basement']
                data_HT.loc[i,'Pool'] = listing['pool']
                data_HT.loc[i,'Extras'] = listing['extras']
                data_HT.loc[i,'Description'] = listing['description']
                data_HT.loc[i,'MLS'] = listing['mlsNum']
                data_HT.loc[i,'Type'] = listing['type']
                data_HT.loc[i,'Link'] = url
                data_HT.loc[i,'Neighbourhood'] = listing['neighbourhoodName']
                data_HT.loc[i,'Balcony'] = listing['misc']['balcony']
                data_HT.loc[i,'Kitchens'] = listing['misc']['kitchens']
                data_HT.loc[i,'ParkingIncluded'] = listing['misc']['parkingIncluded']
                data_HT.loc[i,'ParkingSpaces'] = listing['misc']['parkingSpaces']
                data_HT.loc[i,'HeatIncluded'] = listing['misc']['heatIncluded']
                data_HT.loc[i,'HydroIncluded'] = listing['misc']['hydroIncluded']
                data_HT.loc[i,'ApproxAge'] = listing['misc']['approxAge']
                data_HT.loc[i,'Latitude'] = listing['position']['coordinates'][0]
                data_HT.loc[i,'Longitude'] = listing['position']['coordinates'][1]
                data_HT.loc[i,'Garage'] = listing['garage']
                data_HT.loc[i,'Heat'] = listing['heat']
                data_HT.loc[i,'Heating_Fuel'] = listing['heatingFuel']
                data_HT.loc[i,'AC'] = listing['ac']
                data_HT.loc[i,'Lot_Frontage'] = listing['lotFrontage']
                data_HT.loc[i,'Lot_Depth'] = listing['lotDepth']
                
            i += 1
                
                
        time.sleep(1)
    
    except:
        print('Problem with:', link_num)
        skipped.append(link_num)
        continue   

data_HT#.to_excel('data_HT.xlsx')

In [ ]:
data_HT.to_excel('data_HT.xlsx')

In [158]:
data_HT.shape

(10, 42)